In [ ]:
!pip install tensorflow_text

     |████████████████████████████████| 4.9 MB 10.0 MB/s 


In [ ]:
import os
from glob import glob

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text

In [ ]:
dir_path = 'XFORMAL'
file_path = dir_path + '.zip'
if not os.path.isdir(dir_path):
    if not os.path.isfile(file_path):
        !wget -O XFORMAL.zip "https://docs.google.com/uc?export=download&id=1cF8AXSQ1OZhgIbaIBWc3n8xgj2_prY8C"
    !7z x XFORMAL.zip

    for i in glob('XFORMAL/gyafc_translated/*/*/*/'):
        list_files = os.listdir(i)
        informal_path = os.path.join(i, '1_informal')
        os.mkdir(informal_path)
        formal_path = os.path.join(i, '1_formal')
        os.mkdir(formal_path)
        for j in list_files:
            if 'informal' in j:
                os.rename(
                    os.path.join(i, j),
                    os.path.join(informal_path, j),
                )
            else:
                os.rename(
                    os.path.join(i, j),
                    os.path.join(formal_path, j),
                )
        os.rename(informal_path, os.path.join(i, 'informal'))
        os.rename(formal_path, os.path.join(i, 'formal'))

--2021-11-15 16:39:48--  https://docs.google.com/uc?export=download&id=1cF8AXSQ1OZhgIbaIBWc3n8xgj2_prY8C
Resolving docs.google.com (docs.google.com)... 74.125.142.100, 74.125.142.101, 74.125.142.102, ...
Connecting to docs.google.com (docs.google.com)|74.125.142.100|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-04-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/9shb8och0ov06q1qkp9mpl3hgploao5i/1636994400000/01047558856633083828/*/1cF8AXSQ1OZhgIbaIBWc3n8xgj2_prY8C?e=download [following]
--2021-11-15 16:40:10--  https://doc-04-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/9shb8och0ov06q1qkp9mpl3hgploao5i/1636994400000/01047558856633083828/*/1cF8AXSQ1OZhgIbaIBWc3n8xgj2_prY8C?e=download
Resolving doc-04-4c-docs.googleusercontent.com (doc-04-4c-docs.googleusercontent.com)... 74.125.199.132, 2607:f8b0:400e:c02::84
Connecting to doc-04-4c-docs.googleusercontent.com (doc-04-4c-d

In [ ]:
def get_label(file_path):
    parts = tf.strings.split(file_path, os.path.sep)

    # Note: You'll use indexing here instead of tuple unpacking to enable this 
    # to work in a TensorFlow graph.
    return parts[-2]

In [ ]:
class_names = ['formal', 'informal']

In [ ]:
def labeler(example, example_path):
    return example, tf.argmax(get_label(example_path) == class_names)

In [ ]:
data_path = 'XFORMAL/gyafc_translated/{}/*/{}/*/*'

In [ ]:
BATCH_SIZE = 512
BUFFER_SIZE = 200000

In [ ]:
def create(path_dataset, train=False):
    labeled_data_sets = []

    for file_name in glob(path_dataset):
        lines_dataset = tf.data.TextLineDataset(file_name)
        labeled_dataset = lines_dataset.map(lambda ex: labeler(ex, file_name))
        labeled_data_sets.append(labeled_dataset)
    
    all_labeled_data = labeled_data_sets[0]
    for labeled_dataset in labeled_data_sets[1:]:
        all_labeled_data = all_labeled_data.concatenate(labeled_dataset)
    if train:
        all_labeled_data = all_labeled_data.shuffle(BUFFER_SIZE)
    all_labeled_data = all_labeled_data.batch(BATCH_SIZE)
    return all_labeled_data

In [ ]:
# train_dataset = create(data_path.format('train'), True)
# valid_dataset = create(data_path.format('test'))

In [ ]:
embedding_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3", trainable=True)

In [ ]:
embedding_layer(['hello'])

<tf.Tensor: shape=(1, 512), dtype=float32, numpy=
array([[ 1.24289773e-01,  8.28480534e-03, -1.83482151e-02,
         5.67233469e-03, -1.06630079e-01, -1.12285828e-02,
        -9.37436055e-03,  5.26803844e-02, -7.13109672e-02,
        -1.13984132e-02, -4.09893431e-02, -3.93921398e-02,
         2.86357533e-02, -1.15787508e-02, -7.68727884e-02,
        -6.30869251e-03,  4.21444550e-02,  4.69250139e-03,
         3.84257063e-02, -8.05334002e-03,  3.06332167e-02,
         1.05655655e-01,  9.70294792e-03, -5.48715927e-02,
         7.41304308e-02,  3.04581523e-02,  2.49440204e-02,
        -3.77580873e-03, -2.09636502e-02,  1.75199229e-02,
        -8.98099691e-03, -3.38733047e-02, -3.86001021e-02,
        -7.62207806e-02,  5.28796576e-02, -1.21068269e-01,
        -4.35939096e-02,  2.66841445e-02, -4.29487005e-02,
         3.19739766e-02, -2.41370201e-02,  1.14430971e-02,
        -2.62515619e-02, -3.47538590e-02, -1.58155859e-02,
        -2.07496015e-03, -3.48410830e-02, -1.57976020e-02,
      

In [ ]:
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=[], dtype=tf.string),
        hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3", trainable=True),
        tf.keras.layers.Dense(512, "relu"),
        tf.keras.layers.Dropout(.5),
        tf.keras.layers.Dense(256, "relu"),
        tf.keras.layers.Dropout(.5),
        tf.keras.layers.Dense(1, 'sigmoid'),
    ])
    model.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )
    return model

In [ ]:
langs = ['en', 'fr', 'it', 'pt', 'ru']

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

In [ ]:
for lang in list(reversed(langs)):
    print(lang)
    train_dataset = create(data_path.format(lang, 'train'), True)
    valid_dataset = create(data_path.format(lang, 'test'))

    model = create_model()
    # Create earlystopping callback
    early_stopping_callback = tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy', min_delta=0,
        patience=2, restore_best_weights = True)

    model.fit(
        train_dataset,
        validation_data=valid_dataset,
        epochs=1024,
        callbacks=[early_stopping_callback]
    )

    tune_dataset = create(data_path.format(lang, 'tune'))

    y_test = np.concatenate([y for x, y in tune_dataset], axis=0)

    y_pred = model.predict(tune_dataset)

    test_predict_around = np.around(y_pred)

    result = classification_report(y_test, test_predict_around, digits=6)
    print(result)

ru
Epoch 1/1024
409/409 [==============================] - 48s 85ms/step - loss: 0.3583 - accuracy: 0.8434 - val_loss: 0.4042 - val_accuracy: 0.8240
Epoch 2/1024
409/409 [==============================] - 36s 81ms/step - loss: 0.3059 - accuracy: 0.8641 - val_loss: 0.4180 - val_accuracy: 0.8274
Epoch 3/1024
409/409 [==============================] - 36s 81ms/step - loss: 0.2750 - accuracy: 0.8770 - val_loss: 0.4426 - val_accuracy: 0.8210
Epoch 4/1024
409/409 [==============================] - 36s 82ms/step - loss: 0.2438 - accuracy: 0.8915 - val_loss: 0.4725 - val_accuracy: 0.8151
              precision    recall  f1-score   support

           0   0.761061  0.931262  0.837603     27263
           1   0.895773  0.668937  0.765913     24077

    accuracy                       0.808239     51340
   macro avg   0.828417  0.800100  0.801758     51340
weighted avg   0.824237  0.808239  0.803982     51340

pt
Epoch 1/1024
409/409 [==============================] - 46s 80ms/step - loss: 0.388

In [ ]:
langs = ['en', 'fr', 'it', 'pt', 'ru']
lang_format = '[!{}]*'

In [ ]:
for lang in langs:
    print(lang)
    tune_dataset = create(data_path.format(lang, 'tune'))
    y_test = np.concatenate([y for x, y in tune_dataset], axis=0)

    y_pred = model.predict(tune_dataset)

    test_predict_around = np.around(y_pred)

    result = classification_report(y_test, test_predict_around, digits=6)
    print(result)

en
              precision    recall  f1-score   support

           0   0.838003  0.950042  0.890512     27263
           1   0.933340  0.792042  0.856905     24077

    accuracy                       0.875945     51340
   macro avg   0.885671  0.871042  0.873709     51340
weighted avg   0.882713  0.875945  0.874752     51340

fr
              precision    recall  f1-score   support

           0   0.780935  0.901185  0.836762     27263
           1   0.864480  0.713752  0.781918     24077

    accuracy                       0.813284     51340
   macro avg   0.822708  0.807468  0.809340     51340
weighted avg   0.820115  0.813284  0.811042     51340

it
              precision    recall  f1-score   support

           0   0.757978  0.876499  0.812941     27263
           1   0.830070  0.683100  0.749447     24077

    accuracy                       0.785801     51340
   macro avg   0.794024  0.779800  0.781194     51340
weighted avg   0.791787  0.785801  0.783164     51340

pt
       

In [ ]:
langs = ['en', 'fr', 'it', 'pt', 'ru']
lang_format = '[!{}]*'

In [ ]:
glob(data_path.format(lang_format.format('en'), 'train'))

['XFORMAL/gyafc_translated/pt/Family_Relationships/train/formal/formal',
 'XFORMAL/gyafc_translated/pt/Family_Relationships/train/informal/informal',
 'XFORMAL/gyafc_translated/pt/Entertainment_Music/train/formal/formal',
 'XFORMAL/gyafc_translated/pt/Entertainment_Music/train/informal/informal',
 'XFORMAL/gyafc_translated/fr/Family_Relationships/train/formal/formal',
 'XFORMAL/gyafc_translated/fr/Family_Relationships/train/informal/informal',
 'XFORMAL/gyafc_translated/fr/Entertainment_Music/train/formal/formal',
 'XFORMAL/gyafc_translated/fr/Entertainment_Music/train/informal/informal',
 'XFORMAL/gyafc_translated/ru/Family_Relationships/train/formal/formal_ru',
 'XFORMAL/gyafc_translated/ru/Family_Relationships/train/informal/informal_ru',
 'XFORMAL/gyafc_translated/ru/Entertainment_Music/train/formal/formal_ru',
 'XFORMAL/gyafc_translated/ru/Entertainment_Music/train/informal/informal_ru',
 'XFORMAL/gyafc_translated/it/Family_Relationships/train/formal/formal',
 'XFORMAL/gyafc_trans

In [ ]:
glob(data_path.format('en', 'test'))

['XFORMAL/gyafc_translated/en/Family_Relationships/train/formal/formal',
 'XFORMAL/gyafc_translated/en/Family_Relationships/train/informal/informal',
 'XFORMAL/gyafc_translated/en/Entertainment_Music/train/formal/formal',
 'XFORMAL/gyafc_translated/en/Entertainment_Music/train/informal/informal']

In [ ]:
for lang in langs:
    print(lang)
    train_dataset = create(data_path.format(lang_format.format(lang), 'train'), True)
    valid_dataset = create(data_path.format(lang, 'test'))

    model = create_model()
    # Create earlystopping callback
    early_stopping_callback = tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy', min_delta=0,
        patience=2, restore_best_weights = True)

    model.fit(
        train_dataset,
        validation_data=valid_dataset,
        epochs=1024,
        callbacks=[early_stopping_callback]
    )

    tune_dataset = create(data_path.format(lang, 'tune'))

    y_test = np.concatenate([y for x, y in tune_dataset], axis=0)

    y_pred = model.predict(tune_dataset)

    test_predict_around = np.around(y_pred)

    result = classification_report(y_test, test_predict_around, digits=6)
    print(result)

en
Epoch 1/1024
1634/1634 [==============================] - 145s 81ms/step - loss: 0.3365 - accuracy: 0.8456 - val_loss: 0.4547 - val_accuracy: 0.7674
Epoch 2/1024
1634/1634 [==============================] - 133s 80ms/step - loss: 0.2889 - accuracy: 0.8666 - val_loss: 0.4528 - val_accuracy: 0.7962
Epoch 3/1024
1634/1634 [==============================] - 131s 79ms/step - loss: 0.2554 - accuracy: 0.8820 - val_loss: 0.5339 - val_accuracy: 0.7572
Epoch 4/1024
1634/1634 [==============================] - 132s 79ms/step - loss: 0.2260 - accuracy: 0.8955 - val_loss: 0.6521 - val_accuracy: 0.7635
              precision    recall  f1-score   support

           0   0.863069  0.737960  0.795626     27263
           1   0.745121  0.867425  0.801635     24077

    accuracy                       0.798675     51340
   macro avg   0.804095  0.802693  0.798631     51340
weighted avg   0.807755  0.798675  0.798444     51340

fr
Epoch 1/1024
1226/1226 [==============================] - 107s 78ms/ste

In [ ]:
train_dataset = create(data_path.format(lang, 'train'), True)
valid_dataset = create(data_path.format(lang, 'test'))

In [ ]:
model = create_model()
# Create earlystopping callback
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy', min_delta=0,
    patience=2, restore_best_weights = True)

In [ ]:
model.evaluate

In [ ]:
model.fit(
    train_dataset,
    validation_data=valid_dataset,
    epochs=1024,
    callbacks=[early_stopping_callback]
)

In [ ]:
valid_dataset = create(data_path.format('tune'))

In [ ]:
y_pred = model.predict(np.asarray(X_test))

In [ ]:
test_predict_around = np.around(y_pred)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
confusion_mtx = confusion_matrix(y_test, test_predict_around)
plt.figure(figsize=(10, 8))
cm = confusion_mtx.astype('float') / confusion_mtx.sum(axis=1)[:, np.newaxis]
sns.heatmap(cm, annot=True, fmt='.2f')
plt.xlabel('Prediction')
plt.ylabel('Label')
plt.show()

In [ ]:
result = classification_report(y_test, test_predict_around, digits=6)
print(result)